# Part 1: Data Owner Domain Setup

In this notebook, you will walk through the steps to setup a single-machine Domain node, load some data into it, and create two user accounts:

- *Data Compliance Officer:* someone who has the ability to respond to data-access requests
- *Data Scientist:* someone who will find your data, run an experiment, and then create a data-access request to download their results.

### Step 1: Clone PyGrid
First begin by cloning PyGrid (https://github.com/OpenMined/PyGrid) and cd-ing into the Domain application directory.


```bash
git clone https://github.com/OpenMined/PyGrid.git
cd apps/domain
```

### Step 2: Install Poetry (if you don't have it)

Now we need to ensure that poetry is installed. The following command should work for you (OSX/linux/bashonwindows), but if it doesn't visit the poetry docs (https://python-poetry.org/docs/)

```bash
curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python -
```

### Step 3: Install Poetry Dependencies

If you're following this tutorial you should still be in the "PyGrid/apps/domain" folder of PyGrid. From that folder, run the following command:

```bash
poetry install
```

### Step 4 (Optional): Reset Node first?
If you've run a single-machine PyGrid domain before on this machine, you might have a "nodedatabase.db" file in "PyGrid/Apps/domain/src/nodedatabase.db". If you'd like to refresh your machine from scratch run the following (again assuming your CWD is "PyGrid/apps/domain".

```bash
rm src/nodedatabase.db
```

### Step 5: Launch Domain Node

Now with all your dependencies installed, you're ready to launch the domain node!

```bash
./run.sh --port 5000 --start_local_db
```

### Step 6: Create Root Owner and Login

The node is currently launched in "sleep mode" (you can see by visiting 'http://localhost:5000/'). This means that there isn't a root user yet and you need to declare one.

In [1]:
import sqlite3
cursor = sqlite3.connect('/Users/atrask/Laboratory/openmined/PySyft/packages/grid/apps/domain/src/nodedatabase.db').cursor()

def q(msg):
    cursor.execute(msg)
    return cursor.fetchall()

def db():
    for table in q("SELECT name FROM sqlite_master WHERE type='table';"):
        if table[0] != "group":
            res = q("Select * from " + str(table[0]) + ";")
            if len(res) > 0:
                print()
                print(table[0])                
                print(res)

In [1]:
import syft as sy
import torch as th
import numpy as np

# UNCOMMENT THIS FOR FIRST-TIME SETUP
# sy.login().setup(email="owner@myorg.com", password="owerpwd", domain_name="Trask's Node")

# LOGIN AS ROOT USER
root_domain = sy.login(email="owner@myorg.com", password="owerpwd")

In [3]:
# db()

In [4]:
model = th.nn.Linear(2,1)
model_ptr = model.send(root_domain)

In [5]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 5fe761f397c747019f7a376441761acf>,[],,<class 'torch.nn.modules.linear.Linear'>


In [6]:
root_domain.users.create(
    email="compliance@myorg.com", password="pwd123", role="Compliance Officer"
)

In [7]:
root_domain.store

""


In [9]:
db()


bin_object
[('5fe761f3-97c7-4701-9f7a-376441761acf', b'\n$syft.wrappers.torch.nn.ModuleWrapper\x12\x97\x06\n\x06Linear\x1a(in_features=2, out_features=1, bias=True2\xe2\x05\n6\n\x16syft.lib.python.String\x12\x1c\n\x06weight\x12\x12\n\x10\xc6\xeb\xff\xcd\xd48H\x18\x8b\xdb\xc0\xf0\xa9\x99\xc8\xb6\n4\n\x16syft.lib.python.String\x12\x1a\n\x04bias\x12\x12\n\x10\xe2a\x93\xed\xfa\xf6O!\xb4s\xb8\xad,C[]\x12\xcf\x02\n1syft.wrappers.torch.nn.parameter.ParameterWrapper\x12\x99\x02\n\x94\x02*\x88\x02\xff\xff\xff\xff\xf8\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x1a\x00\x06\x00\x05\x00\x08\x00\x0c\x00\x0c\x00\x00\x00\x00\x04\x04\x00 \x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x00(\x00\x07\x00\x08\x00\x0c\x00\x10\x00\x14\x00\x0e\x00\x00\x00\x00\x00\x00\x03\x8c\x00\x00\x004\x00\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x0

In [5]:
# model_ptr.get()

In [6]:
# db()

In [7]:
# root_domain.store

In [8]:
# root_domain.datasets

# Create and Upload a Few Datasets

We can add datasets to the Domain using our root-user login. Note that unless you've done something special no other user is able to add datasets by default.

In [2]:
x = th.tensor([1,2,3,4]).send(root_domain)

In [3]:
db()

NameError: name 'db' is not defined

In [4]:
xor_inputs = np.array([[0,0],
                       [0,1],
                       [1,0],
                       [1,1]])

xor_input_dataset = root_domain.upload_dataset(data=xor_inputs, 
                                          description="XOR dataset inputs", 
                                          tags=['#xor', '#inputs'])

xor_targets = np.array([[0],
                       [1],
                       [1],
                       [0]])

xor_target_dataset = root_domain.upload_dataset(data=xor_targets, 
                                          description="XOR dataset targets", 
                                          tags=['#xor', '#targets'])

In [8]:
root_domain.datasets

,id,manifest,description,tags,data
0,edf24b85-8446-45e7-93f7-88d35efe6d30,XOR dataset inputs,XOR dataset inputs,"[#xor, #inputs]","[{'name': 'tmp/data_01.csv', 'id': 'aea648ee-1..."
1,7a068408-83d5-4d1e-91a8-6ab5cda82311,XOR dataset targets,XOR dataset targets,"[#xor, #targets]","[{'name': 'tmp/data_01.csv', 'id': 'a9fd788f-c..."


# Create a Data Compliance Officer

Eventually a Data Scientist is going to need some management, particularly when they want to download their statistical results. While this doesn't always need manual review, sometimes it does and it's important we create a user who is capable of reviewing and approving data access requests.

In [9]:
root_domain.roles

,id,name,can_triage_requests,can_edit_settings,can_create_users,can_create_groups,can_edit_roles,can_manage_infrastructure,can_upload_data
0,1,User,False,False,False,False,False,False,False
1,2,Compliance Officer,True,False,False,False,False,False,False
2,3,Administrator,True,True,True,True,False,False,True
3,4,Owner,True,True,True,True,True,True,True


In [21]:
root_domain.users.create(
    email="compliance@myorg.com", password="pwd123", role="Compliance Officer"
)

In [16]:
root_domain.users

,id,email,private_key,verify_key,role,groups
0,1,owner@myorg.com,ae8799746d945cf1dce1f819c74cd35902465f7c303ed6...,7369f4951a6f30143ded8fbeb3bb94968237170a2652b6...,4,[]
1,2,compliance@myorg.com,489c1ad1948172f54ffa10f72aa962b3c9ff4bf195d182...,65ec383b86a4b28b77ca5499a98456a39e0a5281470400...,2,[]


# Done with Part 1!

You may now proceed to Part 2!